In [ ]:
%matplotlib widget
from sys import stdout
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
import nglview
import mdtraj
import pandas
import matplotlib.pyplot as plt

The PDB file for 1YRF contains more information than we need. It holds several possible states of some residues mixed into one file and OpenMM cannot handle this. Most atoms positions are the same for all these states, but a few have an extra `A`, `B`, `C`, ... just after the atom name. The following function splits such a PDB file into multiples ones. Each of these files is suitable for starting an OpenMM simulation.

In [ ]:
def split_pdb(fn_pdb):
    pos = 16
    groups = {}
    counter = 0
    with open(fn_pdb) as f:
        for line in f:
            if line.startswith("ATOM"):
                state = line[pos]
                line = line[:pos] + " " + line[pos+1:]
                groups.setdefault(state, []).append((counter, line))
                counter += 1
    for key, lines_group in groups.items():
        if key == " ":
            continue
        lines_both = lines_group + groups[" "]
        lines_both.sort()
        with open("{}_{}.pdb".format(fn_pdb[:-4], key.lower()), "w") as f:
            for counter, line in lines_both:
                f.write(line)
split_pdb("1yrf.pdb")

In [ ]:
pdb = PDBFile('1yrf_a.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addHydrogens(forcefield)
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy(maxIterations=100)
positions = simulation.context.getState(getPositions=True).getPositions()
with open('init.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, positions, f)

In [ ]:
simulation.reporters = []
simulation.reporters.append(DCDReporter('traj.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars.csv", 100, time=True,
    potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(1000)

In [ ]:
df = pandas.read_csv("scalars.csv")
df.plot(kind='line', x='#"Time (ps)"', y='Potential Energy (kJ/mole)')

In [ ]:
traj = mdtraj.load('traj.dcd', top='init.pdb')
view = nglview.show_mdtraj(traj)
view